# tutorial 24, create ec2 instance with (key pair & security) to login from local machine using ssh

In [6]:
import boto3
ec2_resourc=boto3.resource("ec2")


In [45]:
ec2_resourc.create_instances(ImageId='ami-03657b56516ab7912',
                             InstanceType="t2.micro",
                             MaxCount=1,
                             MinCount=1)

[ec2.Instance(id='i-04b69a02d164ee4ba')]

In [22]:
#create key pair
key_pair=ec2_resourc.create_key_pair(KeyName='key_sample4')


In [8]:
key_value=key_pair.key_material

In [9]:
import io
with io.open("key_sample.pem","w",encoding="utf-8")as f1:
    f1.write(str(key_value))
    f1.close()

In [14]:
#create security group
ec2_client=boto3.client("ec2")

In [15]:
ec2_client.create_security_group(Description='tutorial24 for ec2 ssh access',
      GroupName='samplesg')

{'GroupId': 'sg-0da9083c7f8d0ecd5',
 'ResponseMetadata': {'RequestId': '956102b3-2d0e-4ff4-a678-109683e0e214',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '956102b3-2d0e-4ff4-a678-109683e0e214',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '283',
   'date': 'Thu, 12 Nov 2020 12:44:11 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [16]:
#enable inbound rule for sg
ec2_client.authorize_security_group_ingress(
      GroupId='sg-0da9083c7f8d0ecd5',
      IpPermissions=[
          {
              'FromPort': 0,
              'IpProtocol': '-1',
              'IpRanges': [
                  {
                      'CidrIp': '0.0.0.0/0',
                      
                  },
              ],
              'ToPort': 65536,
          }
      ]
              )

{'ResponseMetadata': {'RequestId': 'a48c5484-a762-495a-af70-8f41132da13e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a48c5484-a762-495a-af70-8f41132da13e',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '259',
   'date': 'Thu, 12 Nov 2020 12:45:46 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

# create ec2 instance with ssm agent ,security group , iampolicy and key pair

In [18]:
ec2_resourc.create_instances(ImageId='ami-03657b56516ab7912',
      InstanceType='t2.micro',
    KeyName='key_sample',
    MaxCount=1,
      MinCount=1,
        SecurityGroupIds=[
          'sg-0da9083c7f8d0ecd5',
      ],
        IamInstanceProfile={
          'Name': 'ssm-role-ec2'
      },
    UserData="""
    #!/bin/bash
cd /tmp
sudo yum install -y https://s3.amazonaws.com/ec2-downloads-windows/SSMAgent/latest/linux_amd64/amazon-ssm-agent.rpm
sudo systemctl enable amazon-ssm-agent
sudo systemctl start amazon-ssm-agent
    """)

[ec2.Instance(id='i-067f8e07acafa63bb')]